In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK073289,Karma Resorts,Other,Past Guests,Karma Kandara,PF311272,Ms,Yaojia,Chen,Female,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KK073488,Karma Resorts,Other,Past Guests,Karma Kandara,PF311272,Ms,Yaojia,Chen,Female,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KK073880,Karma Resorts,Other,Past Guests,Karma Kandara,PF312914,Mr.,Kevin Ka Yang,Cheng,Male,...,2025-05-04 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
3,KK073950,Karma Resorts,Other,Past Guests,Karma Kandara,PF313092,Mr,Fraser,Wotzke,Male,...,2025-05-05 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
4,KK074070,Karma Resorts,Other,Past Guests,Karma Kandara,PF313492,Mr.,Luka Michael,Lewis,Male,...,2025-05-07 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,7,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,KK070767,Karma Resorts,Other,Past Guests,Karma Kandara,PF305655,Mr,Varun,Sharma,Unknown,...,2025-05-04 00:00:00,2,OTA,OTA,RETAIL,3,0,0,Email Only,NO
85,KK071046,Karma Resorts,Other,Past Guests,Karma Kandara,PF306291,Mr,Shekhar,Sharma,Unknown,...,2025-05-04 00:00:00,2,TO,TO,RETAIL,4,0,0,Email Only,NO
86,KK072052,Karma Resorts,Other,Past Guests,Karma Kandara,PF308301,<NA>,Minh,Nguyen,Unknown,...,2025-05-08 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email Only,NO
87,KK069670,Karma Resorts,Other,Past Guests,Karma Kandara,PF303506,Ms,Cindy,Dong,Unknown,...,2025-05-06 00:00:00,4,OTA,OTA,RETAIL,2,0,0,Email Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
1,KK073488,Karma Resorts,Other,Past Guests,Karma Kandara,PF311272,Ms,Yaojia,Chen,Female,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
13,KK073510,Karma Resorts,Other,Past Guests,Karma Kandara,PF312010,Mr,Shen,Li,Male,...,2025-05-05 00:00:00,5,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
17,KK073087,Karma Resorts,Other,Past Guests,Karma Kandara,PF309894,Mr,Haiqi,Yang,Male,...,2025-05-05 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
18,KK073147,Karma Resorts,Other,Past Guests,Karma Kandara,PF309894,Mr,Haiqi,Yang,Male,...,2025-05-05 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
21,KK073111,Karma Resorts,Other,Past Guests,Karma Kandara,PF310743,Mr,Yaojia,Chen,Male,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
22,KK072402,Karma Resorts,Other,Past Guests,Karma Kandara,PF309166,Mr,Haiqi,Yang,Male,...,2025-05-05 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
25,KB050359,Karma Resorts,Other,Past Guests,Karma Bavaria,PF221133,Herr,Michael,Bosse,Male,...,2025-05-05 00:00:00,4,STAFF_PRIV,STAFF,COMP,1,0,0,Email and Phone,NO
33,KB050397,Karma Resorts,Other,Past Guests,Karma Bavaria,PF095400,Frau,Stefanie,Kuschel,Female,...,2025-05-05 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
39,KB049985,Karma Resorts,Other,Past Guests,Karma Bavaria,PF311544,Frau,Luise,Badenhoop,Female,...,2025-05-04 00:00:00,3,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
41,KB048866,Karma Resorts,Other,Past Guests,Karma Bavaria,PF295139,Firma,Karma,Bavaria,Unknown,...,2025-05-04 14:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK073289,Karma Resorts,Other,Past Guests,Karma Kandara,PF311272,Ms,Yaojia,Chen,Female,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KK073880,Karma Resorts,Other,Past Guests,Karma Kandara,PF312914,Mr.,Kevin Ka Yang,Cheng,Male,...,2025-05-04 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
2,KK073950,Karma Resorts,Other,Past Guests,Karma Kandara,PF313092,Mr,Fraser,Wotzke,Male,...,2025-05-05 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
3,KK074070,Karma Resorts,Other,Past Guests,Karma Kandara,PF313492,Mr.,Luka Michael,Lewis,Male,...,2025-05-07 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,7,0,0,Email and Phone,NO
4,KK070853,Karma Resorts,Other,Past Guests,Karma Kandara,PF305787,Miss,Mia Teeor,Scaturchio,Female,...,2025-05-06 00:00:00,4,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
5,KK070204,Karma Resorts,Other,Past Guests,Karma Kandara,PF247717,Mrs,Rachael,Matthews,Female,...,2025-05-05 00:00:00,2,HOUSE_USE,V_OWNER,COMP,7,0,0,Email and Phone,NO
6,KK074103,Karma Resorts,Other,Past Guests,Karma Kandara,PF140764,Mr,Anthony,Smith,Male,...,2025-05-06 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
7,KB050108,Karma Resorts,Other,Past Guests,Karma Bavaria,PF313335,Mr,Stoyan,Avramov,Male,...,2025-05-05 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
8,KK073824,Karma Resorts,Other,Past Guests,Karma Kandara,PF312772,Mr.,Martin,Mecanovski,Male,...,2025-05-07 00:00:00,9,HOUSE_USE,V_OWNER_NRB,COMP,6,0,0,Email and Phone,NO
9,KK069262,Karma Resorts,Other,Past Guests,Karma Kandara,PF227672,Mr.,Eric Rishi,Sami,Male,...,2025-05-04 00:00:00,4,WEDDING,WEDDING,GROUP,6,0,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
6,KK074103,Karma Resorts,Other,Past Guests,Karma Kandara,PF140764,Mr,Anthony,Smith,Male,...,2025-05-06 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
23,KB045606,Karma Resorts,Other,Past Guests,Karma Bavaria,PF213986,Ms,Sabine,Reiner,Female,...,2025-05-04 00:00:00,4,COMP,GER_OWE,COMP,3,0,0,Email and Phone,NO
26,KB047015,Karma Resorts,Other,Past Guests,Karma Bavaria,PF299349,Herr,Uwe,Grassl,Male,...,2025-05-04 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
40,KSM014297,Karma Resorts,Other,Past Guests,Karma St Martins,PF067859,Miss,Lauren,Wall,Unknown,...,2025-05-07 00:00:00,2,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB045603,Karma Resorts,Other,Past Guests,Karma Bavaria,PF095400,Frau,Stefanie,Kuschel,Female,...,2025-05-04 00:00:00,1,COMP,GER_OWE,COMP,2,1,0,Email and Phone,NO
1,KB047879,Karma Resorts,Other,Past Guests,Karma Bavaria,PF303272,Frau,Elisabeth,Feser,Female,...,2025-05-04 00:00:00,3,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KB050435,Karma Resorts,Other,Past Guests,Karma Bavaria,PF313455,Herr,Sandro,Reuter,Male,...,2025-05-04 00:00:00,2,OTA,OTA,RETAIL,2,0,2,Email and Phone,NO
3,KB044929,Karma Resorts,Other,Past Guests,Karma Bavaria,PF288794,Herr,Florian,Oberkofler,Male,...,2025-05-04 00:00:00,3,OTA,OTA,RETAIL,5,1,0,Email and Phone,NO
4,KK072826,Karma Resorts,Other,Past Guests,Karma Kandara,PF309965,Ms,Siqing,Guan,Female,...,2025-05-04 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
5,KK073509,Karma Resorts,Other,Past Guests,Karma Kandara,PF312010,Mr,Shen,Li,Male,...,2025-05-04 00:00:00,4,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
6,KB047515,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301350,Herr,Jochen,Weber,Male,...,2025-05-04 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
7,KK069262,Karma Resorts,Other,Past Guests,Karma Kandara,PF227672,Mr.,Eric Rishi,Sami,Male,...,2025-05-04 00:00:00,4,WEDDING,WEDDING,GROUP,6,0,0,Email and Phone,NO
8,KK073199,Karma Resorts,Other,Past Guests,Karma Kandara,PF311046,Mr.,Varun,Sharma,Male,...,2025-05-04 00:00:00,2,OTA,OTA,RETAIL,5,0,0,Email and Phone,NO
9,KK073822,Karma Resorts,Other,Past Guests,Karma Kandara,PF313359,Miss,Margarita,Ivanova,Female,...,2025-05-04 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_19236\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB045603,Karma Resorts,Other,Past Guests,Karma Bavaria,PF095400,Frau,Stefanie,Kuschel,Female,...,2025-05-04,1,COMP,GER_OWE,COMP,2,1,0,Email and Phone,NO
1,KB048003,Karma Resorts,Other,Past Guests,Karma Bavaria,PF295139,Firma,Karma,Bavaria,Unknown,...,2025-05-04,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
2,KB047519,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301376,Frau,Katja,Popp,Female,...,2025-05-04,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
3,KK073224,Karma Resorts,Other,Past Guests,Karma Kandara,PF311096,Mr,Shekhar,Sharma,Male,...,2025-05-04,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK073284,Karma Resorts,Other,Past Guests,Karma Kandara,PF311237,Mr.,Jason Cimmaron,Simon,Male,...,2025-05-04,7,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
5,KK073822,Karma Resorts,Other,Past Guests,Karma Kandara,PF313359,Miss,Margarita,Ivanova,Female,...,2025-05-04,2,HOUSE_USE,V_OWNER_NRB,COMP,2,2,0,Email and Phone,NO
6,KK073199,Karma Resorts,Other,Past Guests,Karma Kandara,PF311046,Mr.,Varun,Sharma,Male,...,2025-05-04,2,OTA,OTA,RETAIL,5,0,0,Email and Phone,NO
7,KK073880,Karma Resorts,Other,Past Guests,Karma Kandara,PF312914,Mr.,Kevin Ka Yang,Cheng,Male,...,2025-05-04,4,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
8,KB047515,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301350,Herr,Jochen,Weber,Male,...,2025-05-04,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
9,KK073509,Karma Resorts,Other,Past Guests,Karma Kandara,PF312010,Mr,Shen,Li,Male,...,2025-05-04,4,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 89
Jumlah Data Yang Duplicate : 37
Jumlah Data Setelah Hapus Duplicate : 52
Jumlah Data Yang Termasuk Member: 4
Jumlah Data Setelah Hapus Member : 48
Jumlah Data Akhir : 48


In [19]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)